In [1]:
!pip install gym==0.26.2 gym[atari]==0.26.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.1 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install autorom[accept-rom-license]
!pip install gym[atari,accept-rom-license]==0.26.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=90414f87d6fbe61c39e510c96d3f9d10540bf09a3383547e7f7c771f6d6fb46d
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1


In [3]:
! wget http://www.atarimania.com/roms/Roms.rar
! python -m atari_py.import_roms ~/.kaggle/working/

--2024-06-05 09:58:40--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 212.83.163.6
Connecting to www.atarimania.com (www.atarimania.com)|212.83.163.6|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.atarimania.com/roms/Roms.rar [following]
--2024-06-05 09:58:40--  https://www.atarimania.com/roms/Roms.rar
Connecting to www.atarimania.com (www.atarimania.com)|212.83.163.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19612325 (19M) [application/octet-stream]
Saving to: 'Roms.rar'

Roms.rar            100%[===================>]  18.70M  8.41MB/s    in 2.2s    

2024-06-05 09:58:43 (8.41 MB/s) - 'Roms.rar' saved [19612325/19612325]

/opt/conda/bin/python: Error while finding module specification for 'atari_py.import_roms' (ModuleNotFoundError: No module named 'atari_py')


In [4]:
! ale-import-roms ~/.kaggle/working/

Path /root/.kaggle/working doesn't exist.


## Downloading an environment, resolving lots of problems is like 20% of the time

In [5]:
import gym
import matplotlib.pyplot as plt
import os
import random
import torch
import copy
import torch.nn as nn
import numpy as np
from torch.optim import Adam
from itertools import count
from collections import deque, namedtuple
os.environ["SDL_VIDEODRIVER"] = "dummy"
from gym.wrappers.monitoring import video_recorder

In [6]:
env = gym.make("BreakoutNoFrameskip-v4", render_mode='rgb_array')

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [7]:
device = 'cpu'

## Start with preprocessing states

In [9]:
from gym.core import ObservationWrapper
import cv2
from torchvision.transforms.functional import resize
from torchvision.transforms.functional import crop
# will use this built-in class

In [10]:
class Preprocessing(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)

        self.image_size = (64, 64)
        self._gray_scale_rule = torch.tensor([[0.8, 0.1, 0.1]], dtype=torch.float32).to(device).T.unsqueeze(1).to(device)

    def _gray_scale(self, img):
        return torch.matmul(img, self._gray_scale_rule.reshape(-1, 1))

    def observation(self, img):
        img = img[30:200, 6:154]
        img = cv2.resize(img, self.image_size, interpolation=cv2.INTER_AREA)
        img = torch.tensor(img, dtype=torch.float32).to(device)
        img = self._gray_scale(img).squeeze(-1)
        img = torch.tensor(img, dtype=torch.float32).to(device).unsqueeze(0) / 255
        return img



## Now incorporate some rules and preprocessing:

In [11]:
# code from openai/baselines but remaked due to version of gym

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, info, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, info, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs, info

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info, _ = self.env.step(action)
        self.was_real_done = done
        print(done)
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info, _

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            print(self.was_real_done)
            obs, info = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, info, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs, info

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info, _ = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info, _

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


In [12]:
from gym.wrappers import FrameStack

def AtariWrap(env):
    env = MaxAndSkipEnv(env, skip=2)
    env = FireResetEnv(env)
    env = ClipRewardEnv(env)
    env = Preprocessing(env)
    env = FrameStack(env, 4)
    return env
env = AtariWrap(env)

In [22]:
def process_observation(obs):
    return torch.tensor(np.array([frame.to('cpu') for frame in obs._frames]), dtype=torch.float32).unsqueeze(0)

## Create a DeepLearning model which will get a sequence of 4 frames

In [21]:
class Agent(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        self.n = n_actions

        self.conv1 = nn.Conv3d(4, 8, kernel_size=(1, 3, 3), stride=2)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv3d(8, 4, kernel_size=(1, 3, 3), stride=2)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv3d(4, 4, kernel_size=(1, 3, 3))
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv3d(4, 2, kernel_size=(1, 2, 2))
        self.relu4 = nn.ReLU()
    

        self.bottleneck = nn.Flatten()

        self.linear1 = nn.Linear(288, 10)
        self.relu5 = nn.ReLU()
        self.linear2 = nn.Linear(10, n_actions)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        
        x = self.bottleneck(x)
        
        x = self.linear1(x)
        x = self.relu5(x)
        x = self.linear2(x)
        
        return x

    def select_action(self, observation, eps_greedy=True, eps=0.2):
        if eps_greedy:
            if random.random() > eps:
                q = self(observation).squeeze(0)
                max_q = torch.argmax(q)
                return max_q
            else:
                return random.randint(0, self.n-1)
        else:
            q = self(observation).squeeze(0)
            max_q = torch.argmax(q)
            return max_q

    def get_q_value(self, state, action):
        if len(action.shape) == 0:
            return self(state)[0][action.item()]
        else:
            action = action.tolist()
            return self(state)[range(0, len(action)), action]

## Custom loss function creation

In [23]:
def compute_q_loss(model, model_target, states, actions, rewards, next_states, gamma, alpha):
    if len(states.shape) != 5:
        states = states.unsqueeze(0)
        next_states = next_states.unsqueeze(0)
    with torch.no_grad():
        Q_target = torch.max(model_target(next_states.to(device)))
        
    first = model.get_q_value(states.to(device), actions)
    second = rewards.to(device) + gamma * Q_target
    return torch.clip(loss_fn(second, first), -1, 1)

In [24]:
def compute_q_loss_terminal(model, states, actions, rewards, alpha):
    first = model.get_q_value(states.to(device), actions)
    second = rewards.to(device)
    return torch.clip(loss_fn(second, first), -1, 1)

## Load model to continue training

In [25]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = True
    
    model.eval()
    
    return model
# model = load_checkpoint('/kaggle/input/dqn/pytorch/ver1/1/checkpoint (3).pth')

## Training process

In [26]:
model = Agent(env.action_space.n).to(device)
loss_fn = nn.MSELoss()

In [31]:
def train(env, model, episodes, eps_start=1, alpha=1e-3, gamma=0.99, buffer_size=100000, batch_size=32, target_model_update=10000):
    n_done = 0
    print("Started training")
    history = []
    
    model_target = copy.deepcopy(model).to(device)
    model.train()
    model.to(device)
    optim = Adam(params=model.parameters(), lr=1e-4)
    
    
    T = 0
    eps = eps_start
    
    while n_done < episodes:
        states_buffer = torch.empty(size=(1, 4, 1, 64, 64), dtype=torch.float32)
        actions_buffer = torch.tensor([], dtype=torch.uint8)
        rewards_buffer = torch.tensor([], dtype=torch.float32)
        next_states_buffer = torch.empty(size=(1, 4, 1, 64, 64), dtype=torch.float32)
        
        total_reward = 0
        
        state, info = env.reset()
        state = process_observation(state)
        state = state.to(device)
        with torch.no_grad():
            action = model.select_action(state, eps=eps)

        for t in count(0, 1):
            T += 1
            next_state, reward, terminated, done, _info = env.step(action)
            next_state = process_observation(next_state).to(device)

            if terminated:
                loss = compute_q_loss_terminal(model, next_state, torch.tensor([action], dtype=torch.uint8), torch.tensor([reward], dtype=torch.float32), alpha)
                optim.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_value_(model.parameters(), 2)
                optim.step()
                
                history += [total_reward]
                if n_done % 10 == 0:
                    print("episode:", n_done, 'completed by:', t, 'reward:', total_reward)
                if T > target_model_update:
                    with torch.no_grad():
                        model_target = copy.deepcopy(model).to(device)
                    T = 0
                eps = np.clip(eps - 1/episodes, 0.1, 1)
                n_done += 1
                break
                            
            states_buffer = torch.cat((states_buffer, state.to('cpu')))
            actions_buffer = torch.cat((actions_buffer, torch.tensor([action], dtype=torch.uint8)))
            rewards_buffer = torch.cat((rewards_buffer, torch.tensor([reward], dtype=torch.float32)))
            next_states_buffer = torch.cat((next_states_buffer, next_state.to('cpu')))
                        
            total_reward += reward
            
            with torch.no_grad():
                next_action = model.select_action(next_state, eps=eps)

            state = next_state
            action = next_action
            
            loss = compute_q_loss(model, model_target, states_buffer[-1], actions_buffer[-1], rewards_buffer[-1], next_states_buffer[-1], gamma, alpha)
            optim.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), 2)
            optim.step()
            
            if (n_done % 10 == 0) and t==100:
                with torch.no_grad():
                    print('q_values:', model(next_state), 'loss:', loss)
            
            if random.random() < 0.02 and len(states_buffer) > batch_size:
                # there seems to be a bug with using state with index 0, no idea why
                samples = np.random.randint(2, len(states_buffer) - 1, size=batch_size)
                loss = compute_q_loss(model, model_target, states_buffer[samples], actions_buffer[samples], rewards_buffer[samples], next_states_buffer[samples], gamma, alpha)
                optim.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_value_(model.parameters(), 2)
                optim.step()
    model.eval()
    return model, history



In [32]:
model, history = train(env=env, model=model, episodes=1000, eps_start=0.9, alpha=1e-3, gamma=0.99, buffer_size=10000, batch_size=32, target_model_update=10000)

Started training


/tmp/ipykernel_33/3927676706.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img, dtype=torch.float32).to(device).unsqueeze(0) / 255


q_values: tensor([[-0.0068,  0.1477,  0.1399,  0.1319]]) loss: tensor(1.8557e-06, grad_fn=<ClampBackward1>)
episode: 0 completed by: 443 reward: 2.0


KeyboardInterrupt: 

In [ ]:
checkpoint = {'model': Agent(4), 'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')